In [15]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.utils.data as data
import pytorch_lightning as pl
import datasets
import numpy as np

pl.seed_everything(89026614)

text = datasets.load_dataset('tiny_shakespeare')["train"][0]["text"]
#text = open('../../Downloads/simplebooks/simplebooks-2-raw/train.txt', 'r').read()

Global seed set to 89026614
Found cached dataset tiny_shakespeare (/Users/cztomsik/.cache/huggingface/datasets/tiny_shakespeare/default/1.0.0/b5b13969f09fe8707337f6cb296314fbe06960bd9a868dca39e713e163d27b5e)
100%|██████████| 3/3 [00:00<00:00, 557.60it/s]


In [16]:
device = "cpu"
block_size = 100

class MyDataset(data.Dataset):
    def __init__(self, text):
        super().__init__()
        vocab = sorted(set(text))
        self.vocab = vocab
        self.stoi = { ch: i for i, ch in enumerate(vocab) }
        self.itos = { i: ch for i, ch in enumerate(vocab) }
        self.data = torch.tensor([self.stoi[ch] for ch in text], dtype=torch.long).to(device)

    def __len__(self):
        return self.data.size(0) - block_size - 1

    def __getitem__(self, i):
        end = i + block_size
        return self.data[i:end], self.data[i + 1:end + 1]

dataset = MyDataset(text)
train = data.Subset(dataset, range(0, len(dataset) - 500))
test = data.Subset(dataset, range(len(dataset) - 500, len(dataset)))

In [17]:
# adapted from https://github.com/karpathy/minGPT/blob/master/mingpt/model.py
# and https://github.com/karpathy/nanoGPT/blob/master/model.py
# defaults for gpt-mini
class MinGPT(nn.Module):
    def __init__(self, vocab_size, embed_dim=192, num_heads=6, num_layers=6, dropout=0.1):
        super().__init__()
        self.transformer = nn.ModuleDict(dict(
            token_emb = nn.Embedding(vocab_size, embed_dim),
            pos_emb = nn.Embedding(block_size, embed_dim),
            drop = nn.Dropout(dropout),
            layers = nn.Sequential(*[DecoderLayer(embed_dim, num_heads, dropout) for _ in range(num_layers)]),
            norm = nn.LayerNorm(embed_dim),
        ))
        self.lm_head = nn.Linear(embed_dim, vocab_size, bias=False)
    
    def forward(self, x):
        pos = torch.arange(0, x.size(1), dtype=torch.long).unsqueeze(0).to(device)
        x = self.transformer.drop(self.transformer.token_emb(x) + self.transformer.pos_emb(pos))
        x = self.transformer.norm(self.transformer.layers(x))
        return self.lm_head(x)

    @torch.no_grad()
    def generate(self, input_ids, max_new_tokens, top_k=10):
        for _ in range(max_new_tokens):
            out = self(input_ids[:, -block_size:])
            logits = out[:, -1, :]
            v, _ = torch.topk(logits, top_k)
            logits[logits < v[:, [-1]]] = -float("Inf")
            step_res = torch.multinomial(F.softmax(logits, dim=-1), num_samples=1)
            # auto-regression
            input_ids = torch.cat((input_ids, step_res), dim=1)

        return input_ids

class DecoderLayer(nn.Module):
    def __init__(self, embed_dim, num_heads, dropout):
        super().__init__()
        self.ln1 = nn.LayerNorm(embed_dim)
        self.attn = nn.MultiheadAttention(embed_dim, num_heads, dropout, batch_first=True, bias=False)
        self.ln2 = nn.LayerNorm(embed_dim)
        self.mlp = nn.Sequential(
            nn.Linear(embed_dim, 4 * embed_dim),
            nn.GELU(approximate="tanh"),
            nn.Linear(4 * embed_dim, embed_dim),
            nn.Dropout(dropout)
        )
        self.register_buffer("mask", ~torch.tril(torch.ones(block_size, block_size)).to(bool).to(device))

    def forward(self, x):
        B, T, _ = x.shape
        x = self.ln1(x)
        x = x + self.attn(x, x, x, need_weights=False, attn_mask=self.mask[:T, :T])[0]
        x = x + self.mlp(self.ln2(x))
        return x

class Model(pl.LightningModule):
    def __init__(self, vocab_size, lr=0.004):
        super().__init__()
        self.lr = lr
        self.model = MinGPT(vocab_size, embed_dim=192, num_heads=4, num_layers=2, dropout=0.1)
    
    def forward(self, x, y=None):
        logits = self.model(x)
        return logits if y is None else F.cross_entropy(logits.view(-1, logits.size(-1)), y.view(-1), ignore_index=-1)

    def train_dataloader(self):
        return data.DataLoader(train, batch_size=100, num_workers=0, sampler=data.RandomSampler(train, False, 5_000))
    
    def val_dataloader(self):
        return data.DataLoader(test, batch_size=16, num_workers=0)
    
    def training_step(self, batch, batch_idx):
        return self(*batch)

    def validation_step(self, batch, batch_idx):
        loss = self(*batch)
        self.log("test_loss", loss, prog_bar=True, on_step=False, on_epoch=True)
        return loss

    def validation_epoch_end(self, outs):
        with torch.no_grad():
            x = torch.tensor([dataset.stoi[s] for s in "And now "], dtype=torch.long)[None,...].to(device)
            y = self.model.generate(x, 64)[0]
            print("".join([dataset.itos[int(i)] for i in y]))

    def configure_optimizers(self):
        optim = torch.optim.AdamW(self.parameters(), lr=self.lr)
        sched = torch.optim.lr_scheduler.ExponentialLR(optim, gamma=0.999, last_epoch=-1)
        return [optim], [sched]

model = Model(len(dataset.vocab))
trainer = pl.Trainer(gradient_clip_val=1.0, max_epochs=25, enable_progress_bar=True, accelerator="gpu" if device == "cuda" else "cpu")
trainer.fit(model)

GPU available: False, used: False
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
HPU available: False, using: 0 HPUs

  | Name  | Type   | Params
---------------------------------
0 | model | MinGPT | 932 K 
---------------------------------
932 K     Trainable params
0         Non-trainable params
932 K     Total params
3.731     Total estimated model params size (MB)


Epoch 1: 100%|██████████| 82/82 [00:14<00:00,  5.64it/s, loss=2.24, v_num=99, test_loss=2.410]And now st, shy thell and manys of ines with at.

MI:
Thathen wis you so
Epoch 2: 100%|██████████| 82/82 [00:14<00:00,  5.65it/s, loss=2, v_num=99, test_loss=2.110]   And now sith have as ine?

DUKE V:
Now it say, it you, thee fell? that y
Epoch 3: 100%|██████████| 82/82 [00:15<00:00,  5.35it/s, loss=1.85, v_num=99, test_loss=1.970]And now are in thruce.

CLY:
How herefords, thou he am sund mone, so tho
Epoch 4: 100%|██████████| 82/82 [00:29<00:00,  2.80it/s, loss=1.76, v_num=99, test_loss=1.840]And now those and with abost of my brother,
My the with thou here to the
Epoch 5: 100%|██████████| 82/82 [00:22<00:00,  3.69it/s, loss=1.7, v_num=99, test_loss=1.720] And now tear. I sir, my like me sorrour hastage,
And I will theirte dist
Epoch 6: 100%|██████████| 82/82 [00:17<00:00,  4.63it/s, loss=1.65, v_num=99, test_loss=1.670]And now all with the to creath.

RICHARD II:
My groand he sprive! I wer

`Trainer.fit` stopped: `max_epochs=25` reached.


Epoch 24: 100%|██████████| 82/82 [00:17<00:00,  4.75it/s, loss=1.45, v_num=99, test_loss=1.380]


In [18]:
y = model.to(device).model.generate(torch.tensor([dataset.stoi[s] for s in "O God, O God!"]).unsqueeze(0).to(device), 650)
print("".join([dataset.itos[int(i)] for i in y[0]]))

O God, O God!

Gausenger,
Or cause: I low hath it by the for my lame,
Making to sir?
Is you a fair take to field your soul
Still as this pinnt men'd: I'll not this like and
Is fruites and that our dreather with,
As bring arm alliege himself to all the me,
Hene'er they wished!

ISABEO:
And you may the well--
Would then.

GLOUCESTER:
I dacrease,
So first with as them thy saferest broke infice would between with
the lightler, lord in his furs of mine.

POMPEY:
How dot then, that wrow your love, thou shouldst that bear awful aplay,
That were will hath from himsin thy with heil
Is sun facits with her tire wail,
And I adde mine either bear things well
If a both
